On design un chalet en bois, on prend du triple vitrage pour nos 1 fenêtres de 2 m², sur le mur  et pour lisolant on prend de la laine de roche. Pour une pièce de 3x4=12m², donc deux
murs (appelés mur1) de 3m par 3m de hauteur, et deux de 4 par 3 (appelés mur2).

In [ ]:
# thermal conductivity, W/(m·K);
lambdabois=0.06 # wood
lambdaverre=1   # glass
lambdaair=0.024 # air
lambdalaine = 0.034 # whool

# épaisseurs
everre=0.04 
eair=0.1
elaine=0.25
ebois=0.2

# surfaces
Smur1 = 9
Smur2=12
Sfenetre=2

# radiative properties
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  #convection coefficient, W/m² K;
Tin = 20    #temperature inside, K;
σ = σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant

Calcul des différentes résistances thermiques

In [ ]:
#Conduction
Rmur1 = 2*ebois/(lambdabois*(Smur2-Sfenetre))+elaine/(lambdalaine*(Smur2-Sfenetre))
Rfenetre = 3*everre/(lambdaverre*Sfenetre)+2*eair/(lamdaair*Sfenetre)
Rmur2 = 2*ebois/(lambdabois*Smur2)+elaine/(lambdalaine*Smur2)

#Convection
Rconv2 = 2/(h*Smur2)
Rconv1 = 2/(h*Smur1)
Rconvfen = 2/(h*Sfenetre)

# long wave radiation
Tm = Tin + 273   # K, mean temp for radiative exchange

GLW1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * wall['Surface']['Layer_in']
GLW12 = 4 * σ * Tm**3 * Fwg * wall['Surface']['Layer_in']
GLW2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * wall['Surface']['Glass']

Gtot = 1 / (1 / GLW1 + 1 / GLW12 + 1 / GLW2)